#### Managing The Conversation History
- The conversation history is a critical component of the conversation. It is the record of all the messages exchanged
- One important concept to understand when building the chatbots is how to mange conversation history.If left unmanaged,the list of messages will grow unbounded and potentially overflow the context window of the LLM.Therefore,it is important to add a step that limits the size of the messages you are passing in.

In [4]:
import os 
from langchain_core.messages import HumanMessage
from langchain_core.messages import AIMessage
from dotenv import load_dotenv
## loading all the enviroments variables
load_dotenv()
groq_api_key=os.getenv("GROQ_API_KEY")

import os
from langchain_groq import ChatGroq

# Load the Groq API key from an environment variable
groq_api_key = os.environ["GROQ_API_KEY"]

# Create a ChatGroq instance with the specified model and API key
llm = ChatGroq(model="Gemma2-9b-It", groq_api_key=groq_api_key)

# Print the LLM instance (optional)
print(llm)

client=<groq.resources.chat.completions.Completions object at 0x0000021A10163130> async_client=<groq.resources.chat.completions.AsyncCompletions object at 0x0000021A10163D30> model_name='Gemma2-9b-It' groq_api_key=SecretStr('**********')


In [26]:
from langchain_core.messages import SystemMessage,trim_messages
trimmer=trim_messages(
    max_tokens=45,
    strategy="last",
    token_counter=llm,    
    include_system=True,
    allow_partial=False,
    start_on="human"

)

messages=[
    SystemMessage(content="You are a good assistant"),
    HumanMessage(content="Hi!,I am bob"),
    AIMessage(content="hi!"),
    HumanMessage(content="I like vanilla ice cream"),
    AIMessage(content="Nice"),
    HumanMessage(content="Whats 2+2"),
    AIMessage(content="4"),
    HumanMessage(content="Thanks"),
    AIMessage(content="No Problem"),
    HumanMessage(content="Having Fun?"),
    AIMessage(content="Yes!")
]

trimmer.invoke(messages)


[SystemMessage(content='You are a good assistant'),
 HumanMessage(content='I like vanilla ice cream'),
 AIMessage(content='Nice'),
 HumanMessage(content='Whats 2+2'),
 AIMessage(content='4'),
 HumanMessage(content='Thanks'),
 AIMessage(content='No Problem'),
 HumanMessage(content='Having Fun?'),
 AIMessage(content='Yes!')]

In [27]:
from langchain_core.prompts import ChatPromptTemplate,MessagesPlaceholder
prompt=ChatPromptTemplate.from_messages(
    [
        ("system",
         "You are a helpful assistant.Answer all the question to the nest of your ability in {language}"
        ),
        MessagesPlaceholder(variable_name="messages")
    ]
)

In [28]:
from operator import itemgetter
from langchain_core.runnables import RunnablePassthrough

chain=(
    RunnablePassthrough.assign(messages=itemgetter("messages")|trimmer)
    |prompt
    |llm
)

response=chain.invoke(
    {
    "messages":messages + [HumanMessage(content="what ice cream do i like")],
    "language":"English"
    }
)
response.content

"As an AI, I don't have access to your personal information or preferences, so I wouldn't know your favorite ice cream! \n\nWhat's your favorite flavor?  🍦😊\n"

In [29]:
response=chain.invoke(
    {
    "messages":messages + [HumanMessage(content="what math problem did I ask?")],
    "language":"English"
    }
)
response.content

'You asked "Whats 2+2".  😊  \n\n\n\n'

In [30]:
from langchain_community.chat_message_histories import ChatMessageHistory
from langchain_core.chat_history import BaseChatMessageHistory
from langchain_core.runnables.history import RunnableWithMessageHistory

store={}

def get_session_history(session_id:str)->BaseChatMessageHistory:
    if session_id not in store:
        store[session_id]=ChatMessageHistory()
    return store[session_id]
with_message_history=RunnableWithMessageHistory(llm,get_session_history)

In [31]:
## Lets wrap this in the Message History
with_message_history=RunnableWithMessageHistory(
    chain,
    get_session_history,
    input_messages_key="messages"
)
config={"configurable":{"session_id":"chat5"}}

In [38]:
response=with_message_history.invoke(
    {
    "messages":messages + [HumanMessage(content="what is my name?")],
    "language":"English"
    },
    config=config
)
response.content

"As an AI, I don't have access to any personal information about you, including your name.  \n\nDo you want to tell me your name? 😊 \n"

In [39]:
ChatMessageHistory()

InMemoryChatMessageHistory(messages=[])